# Import libraries, set options, connect to DB

In [1]:
# Configuration code for datawrangling
import pandas as pd
import os
import numpy as np
from datetime import datetime
from geocode import geocode
import mapToPoly
from mapToPoly import mapToPoly
pd.set_option('display.max_row', 30000)
import csv

# Configuration code in order to connect to the database
from sqlalchemy import create_engine, exists
from sqlalchemy.orm import sessionmaker
from database_setup_cloud import MixpanelMap, Base

passWord = os.environ['totagoPassWord']
# This commented out one was how I connected to the remote database
DATABASE_URI = 'postgres://awsuser:' + passWord + '@totago-staging-06-28-19.cjtqfbi6mrth.us-west-2.rds.amazonaws.com:5432/totago'

# This code was how I connected to the local database
# DATABASE_URI = 'postgres+psycopg2://maxcarey:' + passWord + '@localhost:5432/totago'
engine = create_engine(DATABASE_URI)

#engine = create_engine('sqlite:///totagoData.db')

# Bind the engine to the metadata of the Base class so that the
# declaratives can be accessed through a DBSession instance
Base.metadata.bind = engine

DBSession = sessionmaker(bind=engine)

session = DBSession()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


# Read in data as pandas data frame, selecting only certain fields

In [2]:
fields = ['distinct_id', 'numItinerariesReturned', 'departureDate', 'startFromLocation', 'selectedDestination_id', 'selectedDestination_name', 'time', 'user_id']

In [3]:
df = pd.read_csv('modified_iten.csv', usecols = fields, dtype={"selectedDestination_id" : "str"})

# Wrange field: destinationIDs

In [4]:
# Replace all of the NAs for destinationIDs with 0
df.selectedDestination_id.fillna(0, inplace = True)

## Remove the 2 cases where the string says null
## Great tutorial here: https://www.youtube.com/watch?v=2AFGPdNn4FM
df = df[df.selectedDestination_id != 'null']

## Convert destinationIDs column to an integer value
df['selectedDestination_id'] = df.selectedDestination_id.astype(int)

# Wrangle field: numItenerariesReturned

In [5]:
# Replace all of the NAs for numItinerariesReturned with 1
df.numItinerariesReturned.fillna(1, inplace = True)

# Convert from float to integer
df['numItinerariesReturned'] = df.numItinerariesReturned.astype(int)

# Select, only observatiosn where this field is greater than 0 (now that the NAs are gone)


# Wrangle Field: Destination Name

In [6]:
#Convert this field to an integer replacing all NA's with zero
# This gets rid of the trailing zeros
df.selectedDestination_name.fillna("", inplace = True)

# Wrangle Field: departureDate

In [7]:
#Convert destinationIDs column to an integer value
# It looks like there were some complex rows being held in here before, I thought that when df.dtypes returned object that
# meant string but apprently not
df['departureDate'] = df.departureDate.astype(str)

print("number of rows before removal of anamoulous departureDate cases")
print(len(df))

# IT looks like there are some cases where this field is blank, says nan, is in format 24503, or in format "masked" 
# We need to remove these cases from the data frame
# I can see that some blank rows are still printed out.
df = df[df.departureDate != '']
df = df[df.departureDate != 'nan']
df = df[df.departureDate != '24503']
df = df[df.departureDate != '[masked]']

print("number of rows after removal of anamoulous departureDate cases")
print(len(df))

# Create a function extractDate that extracts the first ten characters of an input string
def extractDate(dateString):
    extractedDate = dateString[0:10]
    if len(extractedDate) < 10:
        print(extractedDate)
    return extractedDate

''' Code to test if the extractDate function works

# Apply this function to create  a new column
df['departureDateFixed'] = df.departureDate.apply(extractDate)

cols = ['distinct_id', 'departureDate', 'departureDateFixed', 'numItinerariesReturned', 'selectedDestination_id', 'selectedDestination_name', 'startFromLocation']

df = df[cols]
'''

# Override departure date extracting all of the null time stamps
df['departureDate'] = df.departureDate.apply(extractDate)


# Convert departure date into a time object in pandas
#See here: https://stackoverflow.com/questions/26763344/convert-pandas-column-to-datetime
# Though this actually might not need to be done
#df['departureDate'] = df.departureDate.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))




#df['departureDate'] = datetime.strptime(df['departureDate'], '%Y-%m-%-d')  
#df['departureDate'] = pd.to_datetime(df['departureDate'], format = '%Y-%m-%-d')



number of rows before removal of anamoulous departureDate cases
35772
number of rows after removal of anamoulous departureDate cases
35764


# Wrangle Field: distinctID

In [8]:
# Create a coloumn that combines the unix time stamp with distinct_id so that we have a primary key for database
df["primary_key"] = df["distinct_id"] + "-" + df["time"].map(str)
vc = df.primary_key.value_counts()
unique_keys = df.primary_key.unique()

# Wrangle user_id field

In [9]:
df.dtypes

departureDate                object
distinct_id                  object
numItinerariesReturned        int64
selectedDestination_id        int64
selectedDestination_name     object
startFromLocation            object
time                          int64
user_id                     float64
primary_key                  object
dtype: object

In [10]:
# This gets rid of the railing zeros and all of the nas are just
# blank

df['user_id'] = df['user_id'].fillna(0).astype(np.int64)

#Convert to string to be consistent with other fields in database
df['user_id'] = df['user_id'].astype(str)


# Create a subset of the datle with sample method to test geocode and database entry logic

In [11]:
# Create a random sample of the database, these entries will be added to the database in the next section
sampleDf = df.tail(2000)

# Output this random sample
sampleDf.head(len(sampleDf))

,departureDate,distinct_id,numItinerariesReturned,selectedDestination_id,selectedDestination_name,startFromLocation,time,user_id,primary_key
33772,2019-07-06,16c1342304e5ac-06725f44a3af27-37607c04-1aeaa0-...,0,1154,Little Si,"47.5375674,-122.280044",1563666009,0,16c1342304e5ac-06725f44a3af27-37607c04-1aeaa0-...
33773,2019-07-06,16c1342304e5ac-06725f44a3af27-37607c04-1aeaa0-...,0,1154,Little Si,"47.5375678,-122.2800071",1563666023,0,16c1342304e5ac-06725f44a3af27-37607c04-1aeaa0-...
33774,2019-07-21,16c1342304e5ac-06725f44a3af27-37607c04-1aeaa0-...,1,1154,Little Si,"47.537557299999996,-122.2800405",1563666092,827,16c1342304e5ac-06725f44a3af27-37607c04-1aeaa0-...
33775,2019-07-21,16c12bbf7a81da-053425a67542d98-734c1551-2c600-...,1,1150,Mount Si,"47.62111585177648,-122.32002218235384",1563667027,827,16c12bbf7a81da-053425a67542d98-734c1551-2c600-...
33776,2019-07-21,16c13524f2d210-098bd6a561851a-2e694c35-3d10d-1...,1,131,Grouse Grind,"Holdom SkyTrain Station, Holdom Ave at Loughee...",1563667104,0,16c13524f2d210-098bd6a561851a-2e694c35-3d10d-1...
33777,2019-07-21,16c12bbf7a81da-053425a67542d98-734c1551-2c600-...,1,1150,Mount Si,"47.62111585177648,-122.32002218235384",1563668055,827,16c12bbf7a81da-053425a67542d98-734c1551-2c600-...
33778,2019-07-21,16c12bbf7a81da-053425a67542d98-734c1551-2c600-...,1,1150,Mount Si,"47.62111585177648,-122.32002218235384",1563668073,827,16c12bbf7a81da-053425a67542d98-734c1551-2c600-...
33779,2019-07-21,16c12bbf7a81da-053425a67542d98-734c1551-2c600-...,1,1150,Mount Si,"47.62111585177648,-122.32002218235384",1563668168,827,16c12bbf7a81da-053425a67542d98-734c1551-2c600-...
33780,2019-07-21,16c136025fa1b9-0b6ca27e856173-2d6a4d34-2c600-1...,1,1150,Mount Si,"47.62111585177648,-122.32002218235384",1563668230,827,16c136025fa1b9-0b6ca27e856173-2d6a4d34-2c600-1...
33781,2019-07-21,16c13a993e3395-0fb08a6564b8c3-4a526f-13c680-16...,1,1150,Mount Si,"Capitol Hill, Seattle, Washington 98102, Unite...",1563672844,827,16c13a993e3395-0fb08a6564b8c3-4a526f-13c680-16...


## Read in the destination data to allow the possibility to pull the correct names

 


In [12]:
sampleDf.dtypes

departureDate               object
distinct_id                 object
numItinerariesReturned       int64
selectedDestination_id       int64
selectedDestination_name    object
startFromLocation           object
time                         int64
user_id                     object
primary_key                 object
dtype: object

# Loop through the rows in the dataframe, geocode, add entry to database

In [13]:
# Loop through the subsetted pandas data frame

# Uncomment the code below to loop through the the sample data frame
# for index, row in sampleDf.iterrows():

for index, row in sampleDf.iterrows():
  

    # Pull out the primary key into a variable
    testKey = row["primary_key"]
    
    # Check to see if that distinctID is in the data base
    # See this post: https://stackoverflow.com/questions/6587879/how-to-elegantly-check-the-existence-of-an-object-instance-variable-and-simultan?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa
    entryExists = session.query(exists().where(MixpanelMap.distinctkey==testKey)).scalar()

    # If the entry is not in the database
    if not entryExists:
    
        # Get the string to be geocoded
        locationToGeocode = row["startFromLocation"]

        # In the case that the desintaiton ID is zero, this means it was an NA
        # So don't even try to geocode
        
        if row["selectedDestination_id"] != 0:
        
            # Try to run the geocode function that returns a dictionary of information
            try:
                geocodeInfo = geocode(locationToGeocode)
                # If geocoding works, set valid to tre
                valid = True

            # If the geocode function doesn't work set valid to false
            except:
                valid = False
        
        # In the case that the selected Destination ID is 0 then set valid to false
        else:
            valid=False

        # If valid is true create a database entry with information from the dataframe, and the returned geocode informaiton
        if valid:
            
            # Sometimes, such as when a generic city is sent to the geocode() function a geometric center
            # is returned, this means there is no postal code
            
            # In this case, we can set the postalCode to none
            if not 'postalCode' in geocodeInfo:
                geocodeInfo['postalCode'] = "none"

            
            # Mapp the gps coordinates returned to the zip code polygons
            zipCodeInfo = mapToPoly(geocodeInfo['lat'], geocodeInfo['lng'], 'postal')
            
            
            if zipCodeInfo:
                zipCodeMapped = zipCodeInfo[0]
                region = zipCodeInfo[1]
            else:
                zipCodeMapped = 'outsideRegion'
                zipCodeMapped = 'outsideRegion'
            
            barrioInfo = mapToPoly(geocodeInfo['lat'], geocodeInfo['lng'], 'barrio')
            print(barrioInfo)
            
            if barrioInfo:
                barrioMapped = barrioInfo
            else:
                barrioMapped = 'outsideRegion'
                
            ## Get selected Destination Names
            # Pull the selected destination name
            selectedDestinationName = row["selectedDestination_name"]
            
            
            #if not selectedDestinationName:
            #    
            #    key = str(row["selectedDestination_id"])
            #    
            #    if key in destinations:
            #
            #        # Pull the data out from the dictionary that was created in the cell above
            #        newName = destinations[str(row["selectedDestination_id"])]['name']
        #
            #        # Add the new name to the new row
            #        selectedDestinationName  = newName
            #
            #    # In the case that there is destination that corresponds mark
            #    else:
            #    
            #        # TODO: CONSIDER CHANGING THE NAME OF THIS TO SOMETHING ELSE
            #        selectedDestinationName = "DELETED"
            #        # And overwrite valid to false at this point because there is no destination
            #        valid = False
            
            databaseEntry = MixpanelMap(distinctkey=row["primary_key"],
                                      numberitinerariesreturned=row["numItinerariesReturned"],
                                      selecteddestination_id=row["selectedDestination_id"],
                                      selecteddestination_name=selectedDestinationName,
                                      startfromlocation=row["startFromLocation"],
                                      departuredate=row["departureDate"],
                                      # Get data from python dictionary returned from geocode() function
                                      formatted_address=geocodeInfo['formatted_address'],
                                      lat=geocodeInfo['lat'],
                                      lng=geocodeInfo['lng'],
                                      postalcode=geocodeInfo['postalCode'],
                                      postalcodemapped=zipCodeMapped,
                                      barriomapped=barrioMapped,
                                      userid=row["user_id"],
                                      region=region,
                                      valid=valid)
        # If valid is false, just fill in the information that we have from the pandas data frame
        else:
            databaseEntry = MixpanelMap(distinctkey=row["primary_key"],
                                      numberitinerariesreturned=row["numItinerariesReturned"],
                                      selecteddestination_id=row["selectedDestination_id"],
                                      selecteddestination_name=row["selectedDestination_name"],
                                      startfromlocation=row["startFromLocation"],
                                      departuredate=row["departureDate"],
                                      userid=row["user_id"],
                                      valid=valid)

        # Add the the information to a database.    
        session.add(databaseEntry)
        session.commit()
    
    else:
        print("Entry already inside database")

{'formatted_address': '4200 S Othello St, Seattle, WA 98118, USA', 'lat': 47.5377258, 'lng': -122.2803812, 'postalCode': '98118'}
250146
{'formatted_address': '4200 S Othello St, Seattle, WA 98118, USA', 'lat': 47.5377258, 'lng': -122.2803812, 'postalCode': '98118'}
250146
{'formatted_address': '4200 S Othello St, Seattle, WA 98118, USA', 'lat': 47.5377258, 'lng': -122.2803812, 'postalCode': '98118'}
250146
{'formatted_address': '912 E Thomas St, Seattle, WA 98102, USA', 'lat': 47.6212193, 'lng': -122.3201531, 'postalCode': '98102'}
250206
{'formatted_address': '2201 Holdom Ave, Burnaby, BC V5B 3A4, Canada', 'lat': 49.2643868, 'lng': -122.9818363, 'postalCode': 'V5B 3A4'}
None
{'formatted_address': '912 E Thomas St, Seattle, WA 98102, USA', 'lat': 47.6212193, 'lng': -122.3201531, 'postalCode': '98102'}
250206
{'formatted_address': '912 E Thomas St, Seattle, WA 98102, USA', 'lat': 47.6212193, 'lng': -122.3201531, 'postalCode': '98102'}
250206
{'formatted_address': '912 E Thomas St, Seat

None
{'formatted_address': 'Richmond, VA, USA', 'lat': 37.5407246, 'lng': -77.4360481}
None
{'formatted_address': '6200 University Blvd, Vancouver, BC V6T 1Z4, Canada', 'lat': 49.26506999999999, 'lng': -123.250572, 'postalCode': 'V6T 1Z4'}
None
{'formatted_address': 'Seattle, WA, USA', 'lat': 47.6062095, 'lng': -122.3320708}
271849
{'formatted_address': 'Seattle, WA, USA', 'lat': 47.6062095, 'lng': -122.3320708}
271849
{'formatted_address': 'Seattle, WA, USA', 'lat': 47.6062095, 'lng': -122.3320708}
271849
{'formatted_address': '2233 W 13th Ave, Vancouver, BC V6K 2S4, Canada', 'lat': 49.2603816, 'lng': -123.1563323, 'postalCode': 'V6K 2S4'}
KITS
{'formatted_address': '2233 W 13th Ave, Vancouver, BC V6K 2S4, Canada', 'lat': 49.2603816, 'lng': -123.1563323, 'postalCode': 'V6K 2S4'}
KITS
{'formatted_address': '2054 Comox St, Vancouver, BC V6G 1R8, Canada', 'lat': 49.290916, 'lng': -123.1436794, 'postalCode': 'V6G 1R8'}
WE
{'formatted_address': 'Seattle, WA, USA', 'lat': 47.6062095, 'lng':

None
{'formatted_address': '2248 Elgin Ave, Port Coquitlam, BC V3C 2B2, Canada', 'lat': 49.26279290000001, 'lng': -122.7790739, 'postalCode': 'V3C 2B2'}
None
{'formatted_address': '6209 Viewmont St, Tacoma, WA 98407, USA', 'lat': 47.27882640000001, 'lng': -122.5196341, 'postalCode': '98407'}
272015
{'formatted_address': '6209 Viewmont St, Tacoma, WA 98407, USA', 'lat': 47.27882640000001, 'lng': -122.5196341, 'postalCode': '98407'}
272015
{'formatted_address': 'Tacoma, WA, USA', 'lat': 47.2528768, 'lng': -122.4442906}
274674
{'formatted_address': '1506 29th Ave, Seattle, WA 98122, USA', 'lat': 47.6144079, 'lng': -122.2945986, 'postalCode': '98122'}
251187
{'formatted_address': '6288 Stadium Rd, Vancouver, BC V6T 2J9, Canada', 'lat': 49.2549922, 'lng': -123.2447002, 'postalCode': 'V6T 2J9'}
None
{'formatted_address': '3511 Mayfair Ave, Vancouver, BC V6N 2Z2, Canada', 'lat': 49.2371257, 'lng': -123.1825047, 'postalCode': 'V6N 2Z2'}
DS
{'formatted_address': 'Tahoe Metropolitan Planning Org

{'formatted_address': '2248 Elgin Ave, Port Coquitlam, BC V3C 2B2, Canada', 'lat': 49.26279290000001, 'lng': -122.7790739, 'postalCode': 'V3C 2B2'}
None
{'formatted_address': '808 Howell St, Seattle, WA 98101, USA', 'lat': 47.6147059, 'lng': -122.3339779, 'postalCode': '98101'}
343995
{'formatted_address': 'Tacoma, WA, USA', 'lat': 47.2528768, 'lng': -122.4442906}
274674
{'formatted_address': 'Tacoma, WA, USA', 'lat': 47.2528768, 'lng': -122.4442906}
274674
{'formatted_address': 'Tacoma, WA, USA', 'lat': 47.2528768, 'lng': -122.4442906}
274674
{'formatted_address': 'Tacoma, WA, USA', 'lat': 47.2528768, 'lng': -122.4442906}
274674
{'formatted_address': '813 S 7th St, Tacoma, WA 98405, USA', 'lat': 47.2566487, 'lng': -122.4493465, 'postalCode': '98405'}
274674
{'formatted_address': 'Tacoma, WA, USA', 'lat': 47.2528768, 'lng': -122.4442906}
274674
{'formatted_address': 'Vancouver, BC, Canada', 'lat': 49.2827291, 'lng': -123.1207375}
CBD
{'formatted_address': '2555 W 16th Ave, Vancouver, B

{'formatted_address': 'Nanaimo Station, Vancouver, BC V5R 1X5, Canada', 'lat': 49.248321, 'lng': -123.055908, 'postalCode': 'V5R 1X5'}
RC
{'formatted_address': '411 N 90th St, Seattle, WA 98103, USA', 'lat': 47.69400160000001, 'lng': -122.3535755, 'postalCode': '98103'}
250788
{'formatted_address': '12725 33rd Ave NE, Seattle, WA 98125, USA', 'lat': 47.7222191, 'lng': -122.2924536, 'postalCode': '98125'}
271831
{'formatted_address': '150 E Angeleno Ave, Burbank, CA 91502, USA', 'lat': 34.1792162, 'lng': -118.3078009, 'postalCode': '91502'}
761635
{'formatted_address': 'Seattle, WA, USA', 'lat': 47.6062095, 'lng': -122.3320708}
271849
{'formatted_address': 'w Cordova St FS Seymour St, Vancouver, BC V6C 3N3, Canada', 'lat': 49.2855667, 'lng': -123.1115473, 'postalCode': 'V6C 3N3'}
CBD
{'formatted_address': '4840 California Ave SW, Seattle, WA 98116, USA', 'lat': 47.5580055, 'lng': -122.3864097, 'postalCode': '98116'}
344021
{'formatted_address': 'Rainier Ave S & MT Baker TC, Seattle, WA 

...

CBD
{'formatted_address': '10277 City Pkwy #101, Surrey, BC V3T 3K5, Canada', 'lat': 49.1896453, 'lng': -122.8478519, 'postalCode': 'V3T 3K5'}
None
{'formatted_address': '10277 City Pkwy #101, Surrey, BC V3T 3K5, Canada', 'lat': 49.1896453, 'lng': -122.8478519, 'postalCode': 'V3T 3K5'}
None
{'formatted_address': '10277 City Pkwy #101, Surrey, BC V3T 3K5, Canada', 'lat': 49.1896453, 'lng': -122.8478519, 'postalCode': 'V3T 3K5'}
None
{'formatted_address': '10277 City Pkwy #101, Surrey, BC V3T 3K5, Canada', 'lat': 49.1896453, 'lng': -122.8478519, 'postalCode': 'V3T 3K5'}
None
{'formatted_address': 'New Westminster, BC, Canada', 'lat': 49.2057179, 'lng': -122.910956}
None
{'formatted_address': 'Seattle, WA 98101, USA', 'lat': 47.6084921, 'lng': -122.336407, 'postalCode': '98101'}
271849
{'formatted_address': 'Mercer Island, WA 98040, USA', 'lat': 47.5706548, 'lng': -122.2220674, 'postalCode': '98040'}
None
{'formatted_address': 'Mercer Island, WA 98040, USA', 'lat': 47.5706548, 'lng': -122

{'formatted_address': 'University District, Seattle, WA, USA', 'lat': 47.6627771, 'lng': -122.3138767}
272001
{'formatted_address': '6801 S 133rd St, Seattle, WA 98178, USA', 'lat': 47.48362059999999, 'lng': -122.2476724, 'postalCode': '98178'}
None
{'formatted_address': '6801 S 133rd St, Seattle, WA 98178, USA', 'lat': 47.48362059999999, 'lng': -122.2476724, 'postalCode': '98178'}
None
{'formatted_address': '6801 S 133rd St, Seattle, WA 98178, USA', 'lat': 47.48362059999999, 'lng': -122.2476724, 'postalCode': '98178'}
None
{'formatted_address': '6801 S 133rd St, Seattle, WA 98178, USA', 'lat': 47.48362059999999, 'lng': -122.2476724, 'postalCode': '98178'}
None
{'formatted_address': '6801 S 133rd St, Seattle, WA 98178, USA', 'lat': 47.48362059999999, 'lng': -122.2476724, 'postalCode': '98178'}
None
{'formatted_address': '128 W Cordova St, Vancouver, BC V6B 0E6, Canada', 'lat': 49.2830217, 'lng': -123.1080219, 'postalCode': 'V6B 0E6'}
CBD
{'formatted_address': '128 W Cordova St, Vancouv

CBD
{'formatted_address': '498 Robson St, Vancouver, BC V6B 2B5, Canada', 'lat': 49.2800011, 'lng': -123.1177892, 'postalCode': 'V6B 2B5'}
CBD
{'formatted_address': '498 Robson St, Vancouver, BC V6B 2B5, Canada', 'lat': 49.2800011, 'lng': -123.1177892, 'postalCode': 'V6B 2B5'}
CBD
{'formatted_address': '1915 2nd Ave, Seattle, WA 98101, USA', 'lat': 47.6109968, 'lng': -122.3412306, 'postalCode': '98101'}
271849
{'formatted_address': '1915 2nd Ave, Seattle, WA 98101, USA', 'lat': 47.6109968, 'lng': -122.3412306, 'postalCode': '98101'}
271849
{'formatted_address': '2414 1st Ave, Seattle, WA 98121, USA', 'lat': 47.614607, 'lng': -122.3481121, 'postalCode': '98121'}
271808
{'formatted_address': '1021 Howay St, New Westminster, BC V3M 1Z9, Canada', 'lat': 49.207986, 'lng': -122.9270235, 'postalCode': 'V3M 1Z9'}
None
{'formatted_address': 'Richmond-Brighouse Station, 6222 No 3 Rd, Richmond, BC V6Y 2B3, Canada', 'lat': 49.1681432, 'lng': -123.136307, 'postalCode': 'V6Y 2B3'}
None
{'formatted_a

{'formatted_address': '2127 BC-99, Vancouver, BC V6H 3E9, Canada', 'lat': 49.26654809999999, 'lng': -123.1386288, 'postalCode': 'V6H 3E9'}
FAIR
{'formatted_address': '1711 W Meeker St, Kent, WA 98032, USA', 'lat': 47.38172309999999, 'lng': -122.2575706, 'postalCode': '98032'}
None
{'formatted_address': 'Westbound Bedwell Bay Rd @ White Pine Beach Rd, Port Moody, BC V3H 0A3, Canada', 'lat': 49.314355, 'lng': -122.8843499, 'postalCode': 'V3H 0A3'}
None
{'formatted_address': '4435 Slocan St, Vancouver, BC V5R 1Z6, Canada', 'lat': 49.2448506, 'lng': -123.0500652, 'postalCode': 'V5R 1Z6'}
RC
{'formatted_address': '4435 Slocan St, Vancouver, BC V5R 1Z6, Canada', 'lat': 49.2448506, 'lng': -123.0500652, 'postalCode': 'V5R 1Z6'}
RC
{'formatted_address': '2672 Cambridge St, Vancouver, BC V5K 1L5, Canada', 'lat': 49.2863878, 'lng': -123.0501733, 'postalCode': 'V5K 1L5'}
HS
{'formatted_address': 'Vancouver, BC, Canada', 'lat': 49.2827291, 'lng': -123.1207375}
CBD
{'formatted_address': '3843 SW Mar

None
{'formatted_address': '805 146th Ave SE, Bellevue, WA 98007, USA', 'lat': 47.60239989999999, 'lng': -122.1447453, 'postalCode': '98007'}
251060
{'formatted_address': '1755 N Highland Ave, Los Angeles, CA 90028, USA', 'lat': 34.103206, 'lng': -118.3396693, 'postalCode': '90028'}
32059
{'formatted_address': '326 N 76th St, Seattle, WA 98103, USA', 'lat': 47.6841821, 'lng': -122.3540521, 'postalCode': '98103'}
250788
{'formatted_address': '326 N 76th St, Seattle, WA 98103, USA', 'lat': 47.6841821, 'lng': -122.3540521, 'postalCode': '98103'}
250788
{'formatted_address': '4770 32nd Ave S, Seattle, WA 98118, USA', 'lat': 47.55950379999999, 'lng': -122.2904953, 'postalCode': '98118'}
271839
{'formatted_address': 'Seattle, WA, USA', 'lat': 47.6062095, 'lng': -122.3320708}
271849
{'formatted_address': '1010 E Alder St, Seattle, WA 98122, USA', 'lat': 47.6044949, 'lng': -122.3184252, 'postalCode': '98122'}
271869
{'formatted_address': '112 Stewart St, Seattle, WA 98101, USA', 'lat': 47.6108